In [ ]:
# from dotenv import load_dotenv
# load_dotenv()  # take environment variables from .env.
import os
import pprint
import gradio as gr
from openai import OpenAI
pp = pprint.PrettyPrinter(indent=4)

## Create a gradio Web UI

In [ ]:

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

def predict(message, history):
    history_openai_format = []
    for human, assistant in history:
        history_openai_format.append({"role": "user", "content": human })
        history_openai_format.append({"role": "assistant", "content":assistant})
    history_openai_format.append({"role": "user", "content": message})
  
    response = client.chat.completions.create(model='gpt-4o',
    messages= history_openai_format,
    temperature=0.1,
    stream=True)

    partial_message = ""
    for chunk in response:
        if chunk.choices[0].delta.content is not None:
              partial_message = partial_message + chunk.choices[0].delta.content
              yield partial_message

gr.ChatInterface(predict).launch()

## Another Gradio with system prompt, non-stream

In [6]:

from typing import Optional

client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

class Chat:

    def __init__(self, system: Optional[str] = None):
        self.system = system
        self.messages = []
        
        if system is not None:
            self.messages.append({
                "role": "system",
                "content": system
            })

    def prompt(self, content: str) -> str:
          self.messages.append({
              "role": "user",
              "content": content
          })
          response = client.chat.completions.create(
              model="gpt-4o",
              messages=self.messages
          )
          response_content = response.choices[0].message.content
          self.messages.append({
              "role": "assistant",
              "content": response_content
          })
          return response_content

In [ ]:

chat = Chat(system="You are a helpful AI healthcare assistant.")


def respond(message, chat_history):
    bot_message = chat.prompt(content=message)
    chat_history.append((message, bot_message))
    return "", chat_history


with gr.Blocks() as demo:
    chatbot = gr.Chatbot()
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    msg.submit(respond, [msg, chatbot], [msg, chatbot])
    clear.click(lambda: None, None, chatbot, queue=False)

demo.launch(debug=True)